In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from deepface import DeepFace
import glob
from skimage import io
from PIL import Image, ImageEnhance
import json

import time
import pandas as pd
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import warnings
from datetime import datetime
from datetime import timedelta
from datetime import date
import math

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException 
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys

In [2]:
def autoAdjustments_with_convertScaleAbs(img):
    alow = img.min()
    ahigh = img.max()
    amax = 255
    amin = 0

    # calculate alpha, beta
    alpha = ((amax - amin) / (ahigh - alow))
    beta = amin - alow * alpha
    # perform the operation g(x,y)= α * f(x,y)+ β
    new_img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

    return new_img

In [3]:
def restart_page():
    
    driver.get("https://man.kupu.id/#/realNameCertify")

    time.sleep(2)

    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='User Status']"))).click()
    
    time.sleep(1)

    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//li//span[contains(text(),'Under Review')]"))).click()

    driver.find_element(By.XPATH, "//button[@class='el-button el-button--primary el-button--medium']").click()
    
    time.sleep(1)
    
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[(contains(text(),'30/page'))]/../../../../../.."))).click()

    time.sleep(1)

    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[(contains(text(),'100/page'))]"))).click()

In [4]:
old_verification_data = pd.read_csv('verification_data.csv')

In [5]:
old_verification_data['Verification ID'] = old_verification_data['Verification ID'].astype(str)

In [6]:
list_done = old_verification_data['Verification ID'].tolist()

In [9]:
id_list = []
phone_number = []
nik = []
facenet_cosine_result_list = []
facenet_euclidean_result_list = []
vggface_cosine_result_list = []
vggface_euclidean_result_list = []

chromeOptions = webdriver.ChromeOptions()
chromeOptions.add_argument("--start-maximized")

s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s, options=chromeOptions)
actions = ActionChains(driver)
actionChains = ActionChains(driver)


driver.get("https://man.kupu.id/")

WebDriverWait(driver, 100).until(EC.element_to_be_clickable((By.XPATH, "//input[@name='username']"))).send_keys('')

driver.find_element(By.XPATH, "//input[@name='password']").send_keys('')

driver.find_element(By.XPATH, "//button[@class='el-button n_btn pointer el-button--primary el-button--medium']").click()

WebDriverWait(driver, 100).until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Verification')]"))).click()

WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Identity Verification Approval')]"))).click()

time.sleep(1)

WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='User Status']"))).click()

WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//li//span[contains(text(),'Under Review')]"))).click()

driver.find_element(By.XPATH, "//button[@class='el-button el-button--primary el-button--medium']").click()


WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//tr[@class ='el-table__row']/td[not(contains(@class,'is-hidden'))][1]/div[not(.//button)]")))


WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[(contains(text(),'30/page'))]/../../../../../.."))).click()

time.sleep(1)

WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[(contains(text(),'100/page'))]"))).click()

time.sleep(1)

verification_id_list = driver.find_elements(By.XPATH, "//tr[@class ='el-table__row']/td[not(contains(@class,'is-hidden'))][1]/div[not(.//button)]")

phone_number_list = driver.find_elements(By.XPATH, "//tr[@class ='el-table__row']/td[not(contains(@class,'is-hidden'))][3]/div[not(.//button)]")

nik_list = driver.find_elements(By.XPATH, "//tr[@class ='el-table__row']/td[not(contains(@class,'is-hidden'))][5]/div[not(.//button)]")

while len(verification_id_list) != 0:
    
    verification_id_text = []
    
    phone_number_text = []
    
    nik_text = []

    for id in verification_id_list:
        verification_id_text.append(id.text)
        
    for number in phone_number_list:
        phone_number_text.append(number.text)
        
    for nik_number in nik_list:
        nik_text.append(nik_number.text)


    for index, id in enumerate(verification_id_text):
        
        print(id)
        
        if id in list_done:
            
            restart_page()
            
            continue

        else:

            time.sleep(1)

            try:
                
                no = False
                
                while no == False:
                    
                    try:

                        WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "//td[(contains(@class,'is-hidden'))]//div[contains(text(),'"+ id +"')]/../..//td/div/button/span"))).click()

                        time.sleep(1)
                        
                        page_changed = driver.find_elements(By.XPATH, "//button[@class = 'el-button el-button--default el-button--medium']")
                        
                        if page_changed:
                            
                            no = True
                        
                    except:
                        
                        no = False

                        driver.find_element(By.XPATH, "//div[(contains(@class,'el-table__body-wrapper is-scrolling'))]").click()

                        actions.send_keys(Keys.PAGE_DOWN).perform()
                        

                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[@class = 'el-button el-button--default el-button--medium']")))

                time.sleep(1)

                images = driver.find_elements(By.XPATH, "//img[contains(@src,'kupu')]")
                
                if len(images) == 2:
                    img = images[0].get_attribute("src")
                    img2 = images[1].get_attribute("src")
                    img = io.imread(img)
                    img2 = io.imread(img2)
                    img = autoAdjustments_with_convertScaleAbs(img)
                    img2 = autoAdjustments_with_convertScaleAbs(img2)

                    facenet_cosine_result = DeepFace.verify(img, img2,
                                                     model_name = 'Facenet512', 
                                                     detector_backend =  "retinaface",
                                                     distance_metric = "cosine",
                                                     enforce_detection =False)

                    facenet_euclidean_result = DeepFace.verify(img, img2,
                                                     model_name = 'Facenet512', 
                                                     detector_backend =  "retinaface",
                                                     distance_metric = "euclidean_l2",
                                                     enforce_detection =False)

                    vggface_cosine_result = DeepFace.verify(img, img2,
                                                     model_name = "VGG-Face", 
                                                     detector_backend =  "retinaface",
                                                     distance_metric = "cosine",
                                                     enforce_detection =False)

                    vggface_euclidean_result = DeepFace.verify(img, img2,
                                                     model_name = "VGG-Face", 
                                                     detector_backend =  "retinaface",
                                                     distance_metric = "euclidean_l2",
                                                     enforce_detection =False)


                    facenet_cosine_detection_result = facenet_cosine_result['verified']
                    facenet_euclidean_detection_result = facenet_euclidean_result['verified']
                    vggface_cosine_result_result = vggface_cosine_result['verified']
                    vggface_euclidean_result = vggface_euclidean_result['verified']


                    id_list.append(id)
                    phone_number.append(phone_number_text[index])
                    nik.append(nik_text[index])
                    facenet_cosine_result_list.append(facenet_cosine_detection_result)
                    facenet_euclidean_result_list.append(facenet_euclidean_detection_result)
                    vggface_cosine_result_list.append(vggface_cosine_result_result)
                    vggface_euclidean_result_list.append(vggface_euclidean_result)

                else:
                    
                    id_list.append(id)
                    phone_number.append(phone_number_text[index])
                    nik.append(nik_text[index])
                    detection_result = 'No Image'
                    facenet_cosine_result_list.append(detection_result)
                    facenet_euclidean_result_list.append(detection_result)
                    vggface_cosine_result_list.append(detection_result)
                    vggface_euclidean_result_list.append(detection_result)
                
            except:

                pass

            restart_page()
     
    try:
        
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[@class = 'btn-next']"))).click()
    
        time.sleep(2)
    
        verification_id_list = driver.find_elements(By.XPATH, "//tr[@class ='el-table__row']/td[not(contains(@class,'is-hidden'))][1]/div[not(.//button)]")
        
    except:
        
        break
        



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\irvin\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


1515551402360451103
1515544100173660219
1515538949044056117
1515521369587728411
1515511012651581527
1515508154862551117
1514852907357974620
1515451708108062774
1515448991289589804
1515415550418366550
1515405774749839432
1515405389708537898
1515382058481500208
1515378994307547220
1515370646606589996
1515369765257494595
1515355347169718297
1515351993253245017
1515351253566767171
1515348528229658698
1515345520800448556
1515343332581720080
1515329358549102627
1515328253303865414
1515327148515803163
1515324267096064042
1515320508768989233
1515309971913977879
1515303429907038277
1515303380212920369
1515303766357327898
1515299081521086504
1515299075229626428
1500767267855040580
1515293880781258752
1509824893838561288
1515279293574950966
1515266305254113375
1515256994062807088
1515253216651657306
1515252654782693400
1515251572492877878
1515250448641699873
1515247748810158137
1515152824974585872
1515240175482581082
1515239348902703141
1515238105568395270
1515051502258503754
1515232903733850190


In [10]:
verification_data = pd.DataFrame(
    {'Verification ID': id_list,
     'Phone Number': phone_number,
     'NIK': nik,
     'Facenet Cosine': facenet_cosine_result_list,
     'Facenet Euclidean': facenet_euclidean_result_list,
     'Vggface Cosine': vggface_cosine_result_list,
     'Vggface Euclidean': vggface_euclidean_result_list,
    })

In [11]:
new_verification_data = pd.concat([old_verification_data, verification_data])

In [12]:
verification_data.to_csv('verification_data.csv', index = False)

In [15]:
verification_data

,Verification ID,Phone Number,NIK,Facenet Cosine,Facenet Euclidean,Vggface Cosine,Vggface Euclidean
0,1515551402360451103,85772625449,3202161306870003,False,True,True,True
1,1515544100173660219,,3205102207930003,False,True,True,True
2,1515538949044056117,81383931650,3210116104040123,True,True,True,True
3,1515521369587728411,895610643233,6271014308020003,True,True,True,True
4,1515511012651581527,,3275085303890022,False,False,True,True
...,...,...,...,...,...,...,...
75,1477431607425478703,85720868661,3202286301940002,True,True,True,True
76,1476835141573128255,81995482673,5105025404830002,False,False,True,True
77,1472792053686956119,81213033035,3216024503940011,False,False,False,False
78,1465903287999344652,,3175081904971001,True,True,True,True
